In [1]:
from pathlib import Path
import pandas as pd

In [2]:
submissions_path = Path('../data/John Deere Analytics Workshop.csv')
df_submissions = pd.read_csv(submissions_path)

In [3]:
teams_and_pins_path = Path('../data/submission_pins.csv')
df_teams_and_pins = pd.read_csv(teams_and_pins_path)

only keep rows with valid pins

In [4]:
df = df_submissions.merge(df_teams_and_pins, how='inner', left_on='Submission Pin', right_on='pin')

make a datetime column

In [5]:
df['datetime'] = pd.to_datetime(df['Timestamp'])

/var/folders/dj/_mwbb77n38zgmcn3yx6m2ff00000gn/T/ipykernel_48941/3641142192.py:1: FutureWarning: Parsed string "2024/03/27 6:07:21 PM EST" included an un-recognized timezone "EST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df['datetime'] = pd.to_datetime(df['Timestamp'])
/var/folders/dj/_mwbb77n38zgmcn3yx6m2ff00000gn/T/ipykernel_48941/3641142192.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['Timestamp'])
/var/folders/dj/_mwbb77n38zgmcn3yx6m2ff00000gn/T/ipykernel_48941/3641142192.py:1: FutureWarning: Parsed string "2024/03/27 6:13:15 PM EST" included an un-recognized timezone "EST". Dropping unrecognized timezones is deprecated; in a future version this will rais

put most recent submissions first

In [6]:
df.sort_values(by='datetime', inplace=True, ascending=False)

only keep most recent submission per team

In [7]:
df.drop_duplicates(subset='Submission Pin', keep='first', inplace=True)

grade the submissions

In [8]:
df_answers = pd.read_csv('../data/answers.csv')

In [9]:
df['score'] = 0

In [10]:
for question in df_answers['question']:

    correct_answer = df_answers[ df_answers['question'] == question ]['answer'].values[0]

    df_correct = df[ df[ 'Question '+str(question)] == correct_answer ]
    df.loc[ df_correct.index, 'score' ] = df.loc[ df_correct.index, 'score' ]+1

rank winner based on highest score and earliest submission

In [11]:
df.sort_values(by=['score','datetime'], inplace=True, ascending=[False,True])

In [12]:
df

,Timestamp,Submission Pin,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10,team,pin,datetime,score
13,2024/03/27 6:26:05 PM EST,54878528,7368787.0,24.0,73.0,11.0,624.0,34.0,206.0,1998.0,78.0,4.0,6,54878528,2024-03-27 18:26:05,9
14,2024/03/27 6:26:22 PM EST,47716432,7368787.0,24.0,73.0,11.0,624.0,34.0,206.0,1998.0,78.0,4.0,8,47716432,2024-03-27 18:26:22,9
12,2024/03/27 6:34:26 PM EST,37972877,7368787.0,24.0,73.0,11.0,624.0,34.0,206.0,1998.0,78.0,4.0,10,37972877,2024-03-27 18:34:26,9
6,2024/03/27 6:47:40 PM EST,19785474,7368787.0,24.0,73.0,11.0,624.0,34.0,206.0,1998.0,28.0,4.0,9,19785474,2024-03-27 18:47:40,9
10,2024/03/27 6:29:39 PM EST,49287537,7368787.0,215.0,73.0,11.0,624.0,34.0,206.0,1998.0,78.0,4.0,4,49287537,2024-03-27 18:29:39,8
15,2024/03/27 6:30:11 PM EST,78915672,7368787.0,24.0,73.0,11.0,624.0,34.0,206.0,1998.0,78.0,100.0,3,78915672,2024-03-27 18:30:11,8
22,2024/03/27 6:49:38 PM EST,21886983,7368787.0,24.0,73.0,11.0,624.0,NaN,206.0,1998.0,NaN,NaN,2,21886983,2024-03-27 18:49:38,7
16,2024/03/27 6:38:58 PM EST,63343996,7368791.0,24.0,73.0,11.0,624.0,3.0,206.0,1998.0,78.0,100.0,5,63343996,2024-03-27 18:38:58,6
19,2024/03/27 6:49:29 PM EST,22785329,7368787.0,10.0,73.0,11.0,1248.0,34.0,206.0,1998.0,78.0,100.0,1,22785329,2024-03-27 18:49:29,6
23,2024/03/27 6:50:12 PM EST,36255951,7368787.0,23.0,73.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,11,36255951,2024-03-27 18:50:12,3


In [ ]:
df.to_csv('../data/workshop_results.csv')